In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

CSV = 'results_2.3.csv'
PLOTS = 'plots'
os.makedirs(PLOTS, exist_ok=True)

df = pd.read_csv(CSV)
if 'user' not in df.columns:
    df['user'] = 'unknown'

# Preserve threads label and ensure numeric threads column
if 'threads' in df.columns:
    df['threads_label'] = df['threads'].astype(str)
    def threads_to_int(x):
        try: return int(x)
        except: return 1
    df['threads'] = df['threads_label'].apply(threads_to_int)
else:
    df['threads_label'] = '1'

# Timing breakdown per n per user
for user in df['user'].unique():
    user_df = df[df['user'] == user]
    for n in sorted(user_df['n'].unique()):
        d = user_df[user_df['n'] == n].copy()
        if d.empty: continue
        grouped = d.groupby('threads').mean(numeric_only=True).reset_index().sort_values('threads')

        fig, ax = plt.subplots(figsize=(6,3.5))
        ax.plot(grouped['threads'], grouped['time_total'], marker='o')
        ax.set_xlabel('Threads')
        ax.set_ylabel('Time (s)')
        ax.set_title(f'n={n} (user={user})')
        plt.tight_layout()
        plt.savefig(os.path.join(PLOTS, f'timing_n{n}_user_{user}.png'), dpi=200)
        plt.close(fig)

print('Plots saved to', PLOTS)

Plots saved to plots


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

CSV = 'results_2.3.csv'
PLOTS = 'plots'
os.makedirs(PLOTS, exist_ok=True)

df = pd.read_csv(CSV)
if 'user' not in df.columns:
    df['user'] = 'unknown'

# Preserve threads label and ensure numeric threads column
if 'threads' in df.columns:
    df['threads_label'] = df['threads'].astype(str)
    def threads_to_int(x):
        try: return int(x)
        except: return 1
    df['threads'] = df['threads_label'].apply(threads_to_int)
else:
    df['threads_label'] = '1'

# Timing breakdown per n per user
for user in df['user'].unique():
    user_df = df[df['user'] == user]
    for n in sorted(user_df['n'].unique()):
        d = user_df[user_df['n'] == n].copy()
        if d.empty: continue
        grouped_mean = d.groupby('threads').mean(numeric_only=True).reset_index().sort_values('threads')
        grouped_std = d.groupby('threads').std(numeric_only=True).reset_index().fillna(0)
        grouped_std = grouped_std.set_index('threads').reindex(grouped_mean['threads']).reset_index(drop=True)

        fig, ax = plt.subplots(figsize=(6,3.5))
        ax.errorbar(grouped_mean['threads'], grouped_mean['time_total'], yerr=grouped_std['time_total'], marker='o', capsize=4)
        ax.set_xlabel('Threads')
        ax.set_ylabel('Time (s)')
        ax.set_title(f'n={n} (user={user})')
        plt.tight_layout()
        plt.savefig(os.path.join(PLOTS, f'timing_n{n}_user_{user}.png'), dpi=200)
        plt.close(fig)

print('Plots saved to', PLOTS)

Plots saved to plots
